In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


input_size = 784
hidden_size = 128
output_size = 10
model = NeuralNetwork(input_size, hidden_size, output_size)

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
columns = list(train.columns)[1:]

In [4]:
Y = torch.tensor(train['label'].tolist()) 
X = torch.tensor(train[columns].values.tolist()) / 255

In [5]:
dataset_size = len(X)
train_size = int(dataset_size * 0.8)

In [6]:
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

In [7]:

import torch
import torch.optim as optim
torch.seed = 42
# Define hyperparameters
learning_rate = 5e-4
epochs = 10
batch_size = 8


# Create data loaders
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    total += Y_test.size(0)
    correct += (predicted == Y_test).sum().item()
    print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Epoch [1/10], Step [100/4200], Loss: 1.0223
Epoch [1/10], Step [200/4200], Loss: 0.3495
Epoch [1/10], Step [300/4200], Loss: 0.1064
Epoch [1/10], Step [400/4200], Loss: 0.2063
Epoch [1/10], Step [500/4200], Loss: 0.8299
Epoch [1/10], Step [600/4200], Loss: 0.1185
Epoch [1/10], Step [700/4200], Loss: 0.0445
Epoch [1/10], Step [800/4200], Loss: 0.2905
Epoch [1/10], Step [900/4200], Loss: 0.4036
Epoch [1/10], Step [1000/4200], Loss: 0.1867
Epoch [1/10], Step [1100/4200], Loss: 0.3817
Epoch [1/10], Step [1200/4200], Loss: 0.0584
Epoch [1/10], Step [1300/4200], Loss: 0.9072
Epoch [1/10], Step [1400/4200], Loss: 0.2160
Epoch [1/10], Step [1500/4200], Loss: 0.0162
Epoch [1/10], Step [1600/4200], Loss: 0.0234
Epoch [1/10], Step [1700/4200], Loss: 0.1656
Epoch [1/10], Step [1800/4200], Loss: 0.3875
Epoch [1/10], Step [1900/4200], Loss: 0.1220
Epoch [1/10], Step [2000/4200], Loss: 0.1346
Epoch [1/10], Step [2100/4200], Loss: 0.0458
Epoch [1/10], Step [2200/4200], Loss: 0.0849
Epoch [1/10], Step 

In [8]:
test_data = pd.read_csv('test.csv')

In [9]:
test_input = test_data[columns].values.tolist()

In [10]:
with open("res.csv", "w") as f :
    f.write("ImageId,Label\n")
    
for idx, img in enumerate(test_input) :
    img_tensor = torch.tensor(img)
    img_tensor = torch.reshape(img_tensor, (1, -1)) / 255
    output = model(img_tensor)
    _, digit = output.max(dim = 1)
    sample_img = img

    with open("res.csv", "a") as f :
        row = f"{idx+1},{digit.item()}\n"
        f.write(row)